In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from model.CustomDataSet import CustomDataSet
from model.Tokenizer import SimpleTokenizerV1
from environnement.environnement1Str import Environnement1
from environnement.environnement3Str import Environnement3
from environnement.environnement6Str import Environnement6
from environnement.small_loop import small_loop

In [ ]:
class FeedbackPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.input_size = input_size
        self.rnn = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h):
        out, h = self.rnn(x, h)
        return self.fc(out[:, -1, :]), h
    
class InteractionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(InteractionLSTM, self).__init__()
        self.input_size = input_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden=None):
        lstm_out, h = self.lstm(x, hidden)
        output = self.fc(lstm_out[:, -1, :])
        return output, h

In [ ]:
def test_time_training(model, test_seq_act, test_seq_out, action, n_steps=5):
    model.train()
    
    data_set = CustomDataSet(actions=test_seq_act, 
                            outcomes=test_seq_out, 
                            context_lenght=model.input_size,)

    for _ in range(n_steps):  # Ajustement sur le test
        optimizer.zero_grad()
        output = model(test_seq)
        loss = criterion(output, action_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        output = model(test_seq)
    
    return output